<a href="https://colab.research.google.com/github/danielmlow/rallypoint_suicide_detection/blob/main/multimodal_suicide_detector_minimal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model to detect suicidal posts from non-suicidal posts on RallyPoint


### Important: activate GPU in Runtime/Change runtime type, install packages, restart runtime.

### Authors
- Before 2020: Leo (Amazon) and Erik Kastman (RallyPoint and Harvard) built and deployed first models. 
- 2020: Noah re-did models, extended model to include metadata (multimodal) and used a larger dataset. 
- October, 2021: Richard Kuzma reproduced results and fixed dataset by removing duplicate posts. 
- May, 2022: Daniel Low made this minimal script for deployment








In [14]:
!pip install -q transformers

In [18]:
!pip install -q multimodal-transformers

In [13]:
!pip install -q transformers==4.19.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multimodal-transformers 0.2a0 requires transformers>=4.26.1, but you have transformers 4.19.2 which is incompatible.


In [12]:
# !pip uninstall -y multimodal-transformers

In [11]:
!pip install multimodal-transformers

  Using cached multimodal_transformers-0.2a0-py3-none-any.whl (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 12.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.19.2
    Uninstalling transformers-4.19.2:
      Successfully uninstalled transformers-4.19.2


In [7]:
!pip install -q multimodal-transformers==0.1.4a0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 2.2.2 requires transformers<5.0.0,>=4.6.0, but you have transformers 3.1.0 which is incompatible.
flair 0.11.3 requires transformers>=4.0.0, but you have transformers 3.1.0 which is incompatible.


In [3]:
!pip install -q multimodal-transformers==0.1.4a0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 2.2.2 requires transformers<5.0.0,>=4.6.0, but you have transformers 3.1.0 which is incompatible.
flair 0.11.3 requires transformers>=4.0.0, but you have transformers 3.1.0 which is incompatible.


In [2]:
import multimodal_transformers

ModuleNotFoundError: No module named 'multimodal_transformers'

In [16]:
import os
import datetime
import pandas as pd
import numpy as np
import json
import transformers
from multimodal_transformers.data import load_data #was: from multimodal_toolkit.multimodal_transformers.data import load_data
from multimodal_transformers.model import AutoModelWithTabular #was from multimodal_toolkit.multimodal_transformers.model import AutoModelWithTabular
from transformers import AutoConfig, AutoTokenizer, Trainer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, recall_score, f1_score, precision_score
from scipy.special import softmax
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

ModuleNotFoundError: No module named 'multimodal_transformers'

In [5]:
import warnings
warnings.filterwarnings("ignore") #related to sklearn metrics

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
input_dir = '/content/drive/MyDrive/datum/rallypoint_suicide_detection/data/input/' #link to access this directory: https://drive.google.com/drive/folders/1dDdSiYTgSTg8nTKph1wbpKsEwO39VyAR?usp=sharing
output_dir = '/content/drive/MyDrive/datum/rallypoint_suicide_detection/data/output/'
data_path = input_dir+'rp/' #comes from: Noah's Dropbox "RallyPoint Milestone 6 Code/Model notebook/multimodal_toolkit/datasets/rp/"
path_to_checkpoint = input_dir + 'roberta_pretrained_meta_text_num(final)/pred_seed_2/' #comes from: Noah's Dropbox "RallyPoint Milestone 6 Code/Model notebook/Multi-modal Toolkit/logs/"
save_outputs = False #results, plots
config_name = "rp_config(meta).json"
test_dataset_name = "test.csv"


# 1. Load Model

In [8]:
# this took me 30 sec on Google Colab GPU
tokenizer = AutoTokenizer.from_pretrained(path_to_checkpoint)
config = AutoConfig.from_pretrained(path_to_checkpoint)
model = AutoModelWithTabular.from_pretrained(path_to_checkpoint, config=config)

trainer = Trainer(model=model)


# 2. Load test data

In [9]:
load_subset = 10 # or int

test = pd.read_csv(data_path + test_dataset_name)
# test.label = test.label.astype('int8')
if load_subset:
  test = test.sample(load_subset)
test.head()

,id,hash,content,label,dataset,reputation,contact_size,type,ancestry,tags,type_tag_content
651,4548261.0,5a1820eed3230dccb1135b535db1684b,One of the problems for older veterans who com...,0.0,Set 3,0.081236,0.024348,0.0,4547524.0,"[""suicide"",""mental health"",""health"",""caregiver""]",type| comment tag| suicide mental health healt...
1124,1049790.0,88e8a96c7c5192b45e78d97a87641b7f,I like my 38 cal. revolver.,0.0,Set 1,0.093517,0.067501,0.0,1049787.0,"[""firearms and guns"",""shooting"",""self defense""...",type| comment tag| firearms and guns shooting ...
1647,3576546.0,7dc5cc00e94103834bb6b2c00ca48598,Your quote - hard times - is sooo timely and t...,0.0,Set 3,0.059932,0.008166,0.0,3575147.0,"[""motivation"",""inspiration"",""friends"",""faith"",...",type| comment tag| motivation inspiration frie...
405,5488379.0,637230fffce35b7897d9dd56ceaeda28,"Composure or being calm, self-control, master ...",0.0,nock2000,0.012679,0.002248,0.0,5488183.0,"[""motivation"",""inspiration"",""leadership"",""comm...",type| comment tag| motivation inspiration lead...
534,4039616.0,ebde0d8941f18659ef214ff7802ef9cc,"If I did not carry a small .380, then most day...",0.0,Set 5,0.001183,0.002173,0.0,4038834.0,"[""handguns and pistols"",""concealed carry""]",type| comment tag| handguns and pistols concea...


#### 2.1. This the data that is needed


In [10]:
with open(data_path + config_name, "r") as f:
    data_config = dict(json.load(f))

data_config

{'cat_cols': ['type'],
 'label_col': 'label',
 'label_list': ['SITB_ABSENT', 'SITB_PRESENT'],
 'num_cols': ['reputation', 'contact_size'],
 'text_cols': ['type_tag_content']}

These are the variables being used:
- 'label': 0 or 1 (integer; which is SITB_ABSENT or SITB_PRESENT)
- 'reputation' (float between 0 and 1)
- 'contact_size' (float between 0 and 1)
- 'type_tag_content': three string variables concatenated. So we need to import these three variables and concatenate them.
    - 'type': categorical, either: 0 (comment), 0.5 (question) or 1 (StatusUpdate)
    - 'tag': list of strings, e.g., ['health', 'veterans', 'affairs', 'benefits' , 'military', 'family' , 'veterans', 'health', 'administration']
    - 'content: string document 
    - Here's an example : 'type| comment tag| health veterans affairs benefits military family veterans health administration body| That is part of why I complain so loudly. I have been trying desperately to get mental and dental health for YEARS. My teeth are so bad now, that its a miracle I have not died from infections.' 

In [11]:
torch_dataset = load_data(
                            data_df = test,
                            text_cols = data_config['text_cols'], #['type_tag_content']
                            tokenizer=tokenizer,
                            categorical_cols= data_config['cat_cols'], #['type']
                            categorical_encode_type=None,
                            numerical_cols= data_config['num_cols'], #['reputation', 'contact_size']
                            sep_text_token_str=tokenizer.sep_token, #</s>
                            label_col=data_config['label_col'], #in the DF, 0s and 1s. 
                            label_list= data_config['label_list'] #['SITB_ABSENT', 'SITB_PRESENT'] not in DF, just what 0 and 1 are.
)

# 3. Predict

In [15]:
# This took 50 sec for 1712 predictions with Google Colab GPU (2022) or 2 sec for 100 predictions. 
prediction_object = trainer.predict(test_dataset=torch_dataset) #this provides values for 0 and 1
predictions = prediction_object.predictions

# Compare different thresholds to label_ids provided in prediction_object output
predictions_softmax = softmax(predictions, axis=1)
y_pred = np.array([np.argmax(elem) for elem in predictions_softmax]) # threshold = 0.5. Take largest (0.51 over 0.49)
prediction_labels_softmax_score = [np.round(n[label],3) for n,label in zip(predictions_softmax,y_pred)]

# Display minimal info: ID, post, y_pred
test['y_pred'] = y_pred
test['y_pred_softmax'] = prediction_labels_softmax_score
test[['y_pred_softmax_sitb-','y_pred_softmax_sitb+']] = predictions_softmax
test[['y_pred_logit_sitb-','y_pred_logit_sitb+']] = predictions
test[['id','reputation', 'contact_size', 'type_tag_content','y_pred_softmax','y_pred']]

Prediction:   0%|          | 0/2 [00:00<?, ?it/s]

,id,reputation,contact_size,type_tag_content,y_pred_softmax,y_pred
651,4548261.0,0.081236,0.024348,type| comment tag| suicide mental health healt...,0.999,0
1124,1049790.0,0.093517,0.067501,type| comment tag| firearms and guns shooting ...,1.000,0
1647,3576546.0,0.059932,0.008166,type| comment tag| motivation inspiration frie...,0.999,0
405,5488379.0,0.012679,0.002248,type| comment tag| motivation inspiration lead...,0.997,0
534,4039616.0,0.001183,0.002173,type| comment tag| handguns and pistols concea...,1.000,0
1377,3937286.0,0.000711,0.001723,type| comment tag| retirement benifits veteran...,1.000,0
1261,1136429.0,0.128378,0.047723,type| comment tag| transition body| All I want...,1.000,0
23,4224771.0,0.000129,0.000000,type| comment tag| ucmj article 15 humor body|...,1.000,0
602,4056511.0,0.081236,0.024348,type| comment tag| mental health services depr...,0.999,0
1110,3641043.0,0.001923,0.002772,type| comment tag| military career security fo...,1.000,0


In [16]:
ts = datetime.datetime.utcnow().strftime('%y-%m-%dT%H-%M-%S')
test.to_csv(output_dir+f'predictions_{ts}.csv')